# Teste Rápido - YOLO vs CNN

Comparação prática: qual é mais rápido e preciso?

In [ ]:
# Setup
import sys
sys.path.append('../src')

from pathlib import Path
import time
import numpy as np
import torch
from ultralytics import YOLO

from data.dataset import create_dataloaders
from models.cnn_scratch import SimpleCNN, CNNTrainer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

## 1. YOLO Tradicional

Modelo pré-treinado, zero setup.

In [ ]:
# Carregar YOLO
yolo = YOLO('yolov8n.pt')

# Testar velocidade
test_img = str(list(Path('../Coffee/Teste').glob('*.jpg'))[0])

_ = yolo.predict(test_img, device=device, verbose=False)  # warm-up

times = []
for _ in range(20):
    start = time.time()
    _ = yolo.predict(test_img, device=device, verbose=False)
    times.append((time.time() - start) * 1000)

yolo_time = np.mean(times)
print(f"YOLO: {yolo_time:.2f}ms por imagem")

## 2. CNN do Zero

Treinar rápido (5 épocas) e comparar.

In [ ]:
# Carregar dados
dataloaders, _ = create_dataloaders('.', batch_size=16, img_size=(224,224), num_workers=0)

# Criar CNN
model = SimpleCNN(num_classes=2).to(device)
trainer = CNNTrainer(model, device=device)

# Treinar 5 épocas
print("\nTreinando...")
history = trainer.train(dataloaders['train'], dataloaders['val'], epochs=5)

# Testar
y_true, y_pred = trainer.predict(dataloaders['test'])
from sklearn.metrics import accuracy_score
cnn_acc = accuracy_score(y_true, y_pred)

# Velocidade
model.eval()
test_batch = next(iter(dataloaders['test']))
test_input = test_batch[0][:1].to(device)

with torch.no_grad():
    for _ in range(10):
        _ = model(test_input)  # warm-up

times = []
with torch.no_grad():
    for _ in range(100):
        start = time.time()
        _ = model(test_input)
        times.append((time.time() - start) * 1000)

cnn_time = np.mean(times)

print(f"\nCNN Accuracy: {cnn_acc*100:.1f}%")
print(f"CNN Speed: {cnn_time:.2f}ms por imagem")

## 3. Comparação

Quem ganhou?

In [ ]:
print("="*50)
print("RESULTADOS")
print("="*50)
print(f"YOLO:  {yolo_time:.2f}ms")
print(f"CNN:   {cnn_time:.2f}ms")
print(f"\nCNN é {yolo_time/cnn_time:.0f}x mais rápida!")
print(f"CNN Accuracy: {cnn_acc*100:.0f}%")
print("="*50)

## Conclusão

**Para classificação simples (Coffee vs Mountains):**
- CNN é muito mais rápida
- CNN tem boa precisão com poucos dados
- YOLO é melhor quando precisa detectar múltiplos objetos

**Escolha CNN se**: Classificação rápida é prioridade  
**Escolha YOLO se**: Precisa detectar objetos e suas posições